In [1]:
# using bert sequence classifier from hugging face 
# https://huggingface.co/docs/transformers/v4.24.0/en/model_doc/bert#transformers.BertForSequenceClassification
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification 
import pandas as pd
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import f1_score

In [4]:
# dataset = np.array(pd.read_csv('/content/drive/MyDrive/NLP_Project_sem5/processed_data.csv'))
dataset = pd.read_csv('/content/drive/MyDrive/NLP_Project_sem5/processed_data.csv')

In [5]:
print(dataset.shape)
# print(dataset[0,2])

(19826, 3)


In [ ]:
# num_labels = 3
# tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

# model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", problem_type="multi_label_classification", )

In [ ]:
# print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [ ]:
# inputs = tokenizer(dataset['tweet'][0], return_tensors="pt")
# print(inputs.items())
# print(inputs['input_ids'].squeeze(1).shape)
# print(tokenizer.decode(inputs["input_ids"][0]))
# logits = model(**inputs).logits
# print(logits)
# predicted_class_id = logits.argmax().item()
# print(predicted_class_id)
# model.config.id2label[predicted_class_id]



dict_items([('input_ids', tensor([[  101,  1026,  5310, 18442,  1028,  1048,  2213,  7011,  2080,  2023,
          4937,  2318,  6012,  1996,  4485,  2041,  1997,  2033,  2026,  9152,
         23033,  2633,  2288,  2070, 22418,  2026,  9152, 23033,   102]])), ('token_type_ids', tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])), ('attention_mask', tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]]))])
torch.Size([1, 29])
[CLS] < username > lmfao this cat started beating the shit out of me my nigga finally got some pussy my nigga [SEP]
tensor([[-2.5881,  2.9254]], grad_fn=<AddmmBackward0>)
1


'LABEL_1'

In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.labels = [label for label in dataset['label']]
        self.tweets = [tokenizer(tweet, max_length=512, padding='max_length', truncation=True, return_tensors="pt") for tweet in dataset['tweet']]


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tweet = self.tweets[idx]
        label = self.labels[idx]

        return tweet, label

    def getLabels(self):
        return self.labels
    


In [7]:
# sequence classification/regression head on top a linear layer on top of the pooled output of BERT
from transformers import BertModel


class BertSeqClassifier(nn.Module):
    def __init__(self):
        super(BertSeqClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.linear_layer = nn.Linear(768, 3)
        self.soft_max = nn.Softmax(dim=3) 

    def forward(self, input_ids, bert_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=bert_mask, return_dict=False)
        linear_output = self.linear_layer(pooled_output)
        # probs = self.soft_max(linear_output) # already incorporated in crossentropyLoss
        # return probs
        return linear_output





In [13]:
from tqdm import tqdm
from torch.utils.data import DataLoader

def train(model, train_data, test_data, learning_rate, epochs, device):
    model.train()
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    if(device == "cuda"):
        model.cuda()
        loss_function.cuda()

    
    batch_size = 8
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    
    
    for epoch in tqdm(range(epochs)):

        correct_preds = 0
        train_preds = []
        test_preds = []

        for batch, (tweets, labels) in enumerate(train_dataloader):
            
            labels = labels.to(device)
            # print()
            # print("="*40)
            # print()
            # print(tweets["attention_mask"].shape)
            # print()
            output = model(tweets["input_ids"].squeeze(1).to(device), tweets["attention_mask"].to(device))
            # print(f"label shape: {labels.shape}") # should be batch X 1
            loss = loss_function(output, labels)
            # print("output shape: ", output.shape)
            
            # preds = model() 

            # update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # accuracy 
            preds = output.argmax(dim=1)
            # print("preds shape: ", preds.shape)
            # print("labels shape: ", len(labels))

            for i in range(len(labels)):
                # print(i)
                train_preds.append(preds[i].cpu())
                if preds[i] == labels[i]:
                    correct_preds += 1
            
        
        


        # testing code 

        test_correct_preds = 0

        with torch.no_grad():
            for batch, (tweets, labels) in enumerate(test_dataloader):
                labels = labels.to(device)
                # print()
                # print("="*40)
                # print()
                # print(tweets["attention_mask"].shape)
                # print()
                output = model(tweets["input_ids"].squeeze(1).to(device), tweets["attention_mask"].to(device))
                # print(f"label shape: {labels.shape}") # should be batch X 1
                loss = loss_function(output, labels)
                # print("output shape: ", output.shape)
                
                # preds = model() 
                # accuracy 
                preds = output.argmax(dim=1)
                # print("preds shape: ", preds.shape)
                # print("labels shape: ", labels.shape)

                for i in range(len(labels)):
                    # print(i)
                    test_preds.append(preds[i].cpu())
                    if preds[i] == labels[i]:
                        test_correct_preds += 1
        

          
        print(f"\nEpoch: {epoch + 1}, Train Acc: {correct_preds/len(train_data)}, Train Length: {len(train_data)}, Test Acc: {test_correct_preds/len(test_data)}, Test Length: {len(test_data)}")
        print(f"F1 Score Train: {f1_score(train_data.getLabels(), train_preds, average=None)}")
        print(f"Macro F1 Score Train: {f1_score(train_data.getLabels(), train_preds, average='macro')}")
        print(f"F1 Score Test: {f1_score(test_data.getLabels(), test_preds, average=None)}")
        print(f"Macro F1 Score Test: {f1_score(test_data.getLabels(), test_preds, average='macro')}")




In [9]:
!nvidia-smi

Tue Nov 22 10:31:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.current_device()
torch.cuda.get_device_name(torch.cuda.current_device())

cuda


'Tesla T4'

In [14]:
labels_map = {0: "hate?",
              1: "offensive?",
              2: "none?",
              }

# target = torch.empty(3, dtype=torch.long).random_(5)
# print(target)
# print(target.shape)
# print(torch.device("cuda"))


model = BertSeqClassifier()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# print(dataset)
# print(tokenizer)
train_data = CustomDataset(dataset[:10000], tokenizer)
test_data = CustomDataset(dataset[10000:], tokenizer)


train(model, train_data, test_data, 0.00001, 3, device)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 33%|███▎      | 1/3 [02:03<04:06, 123.46s/it]


Epoch: 1, Train Acc: 0.787, Train Length: 1000, Test Acc: 0.819, Test Length: 1000
F1 Score Train: [0.         0.88080582 0.        ]
Macro F1 Score Train: 0.293601939936579
F1 Score Test: [0.         0.90083102 0.08571429]
Macro F1 Score Test: 0.3288484368816779


 67%|██████▋   | 2/3 [04:06<02:03, 123.25s/it]


Epoch: 2, Train Acc: 0.87, Train Length: 1000, Test Acc: 0.892, Test Length: 1000
F1 Score Train: [0.03125    0.79752322 0.11214953]
Macro F1 Score Train: 0.31364091750817397
F1 Score Test: [0.06896552 0.94427434 0.77022654]
Macro F1 Score Test: 0.5944887987201987


100%|██████████| 3/3 [06:09<00:00, 123.24s/it]


Epoch: 3, Train Acc: 0.913, Train Length: 1000, Test Acc: 0.886, Test Length: 1000
F1 Score Train: [0.02597403 0.79043424 0.16766467]
Macro F1 Score Train: 0.32802431066695487
F1 Score Test: [0.18461538 0.94117647 0.75964392]
Macro F1 Score Test: 0.6284785907058555
